# Variational Principle method

This notebook follows (Thijssen, 2013, p. 29). The variational method is a method for deducing upper bounds of energy eigenstates (usually used on the ground state). 

## Theory 

Consider a subset of Hilbert space whose energy expectation values are greater than some eigenstate's energy that we want to find. It can be shown that the min energy expectation value of any superposition of these states is greater than or equal to the eigenenergy we want (see (Thijssen, 2013, p. 30) for the full proof or (Griffiths, 2018, p. 327) for the simple case).

In the linear case this can be reformed into a matrix problem.

If the subset of states is $\{\ket{n}|n=1,2,3,...\}$ (this set doesn't have to be orthonormal), then a general ket takes the form:

$\ket{\phi} = \sum\limits_{i = 1}^{\infty}C_i\ket{i}$

$\implies \bra{\phi}\hat{H}\ket{\phi} = \dfrac{\sum\limits_{i,j = 1}^{\infty}C_i^{*}C_jH_{ij}}{\sum\limits_{i,j = 1}^{\infty}C_i^{*}C_jS_{ij}}$

where $H_{ij} = \bra{i}\hat{H}\ket{j}$ and $S_{ij} = \bra{i}\hat{S}\ket{j}$

Minimising $\bra{\phi}\hat{H}\ket{\phi}$ by calculus gives:

$\sum\limits_{j = 1}^{\infty}C_j(H_{ij} - E S_{ij})$ for all i

This can be rewritten as a matrix equation $\bf{H}\bf{C} = E \bf{S}\bf{C}$

## Linear variational method for the Hydrogen atom

In atomic units the time-independent Schrödinger equation is:

$(\frac{1}{2}\nabla^2 - \frac{1}{r})\psi(\bm{r}) = E\psi(\bm{r})$

Apply the linear variational principle method, where the guessed wavefunction is a linear superposition of:

$\psi_{\alpha_n}(\bm{r}) = e^{-\alpha_n r^2}$

where $\alpha \in \{ \alpha_1 = 13.00773, \alpha_2 = 1.962079, \alpha_3 = 0.444529, \alpha_4 = 0.1219492 \}$

The corresponding matrix elements are:

$S_{nm} = \left( \frac{\pi}{\alpha_n + \alpha_m} \right)^{\frac{3}{2}}$

$H_{nm} = 3\frac{\alpha_n\alpha_m\pi^{\frac{3}{2}}}{(\alpha_n + \alpha_m)^{\frac{5}{2}}} - \frac{2\pi}{\alpha_n + \alpha_m}$

The following program implements this approach.

In [25]:
import numpy as np
import matplotlib.pyplot as plt


In [26]:

#functions that calculate components 
SComponent = lambda alpha_n,alpha_m: (np.pi/(alpha_n + alpha_m))**(3/2)
HComponent = lambda alpha_n,alpha_m: 3*(alpha_n*alpha_m*(np.pi**1.5))/((alpha_n + alpha_m)**(5/2)) - 2*np.pi/(alpha_n + alpha_m)

#values of alpha
alphas = np.array([13.00773, 1.962079, 0.444529, 0.1219492])

#finding matrices as lists
S = [[SComponent(a_n,a_m) for a_m in alphas] for a_n in alphas]
H = [[HComponent(a_n,a_m) for a_m in alphas] for a_n in alphas]

#convert to numpy arrays
S = np.array(S)
H = np.array(H)

#find eigenvalues
eigenvalues, eigenvectors = np.linalg.eig(np.matmul(np.linalg.inv(S),H))

#find min eigenvalue
(groundEnergy,index) = min([(eigenvalue, i) for i, eigenvalue in enumerate(eigenvalues)])

#print out
print(f'Ground energy is {groundEnergy} hartree. The correct value is -0.5 hartree.')

Ground energy is -0.4992784056674857 hartree. The correct value is -0.5 hartree.


## References

Thijseen J., 2013. Computational Physics. Cambridge: Cambridge University Press

Griffiths D., 2018. Introduction To Quantum Mechanics.  Cambridge: Cambridge University Press